# Practical Session 2
### Kernel Methods for Machine Learning

Written by Yunlong Jiao / Romain Menegaux, 21 June 2022

In [ ]:
# setup
import numpy as np

In [ ]:
import sys
print(sys.version)

In [ ]:
import sklearn
from sklearn import linear_model as lm
sklearn.__version__

***
## Tasks

1. Implement (naive) solvers to Ridge Regression, Weighted Ridge Regression and Logistic Ridge Regression (using Iteratively Reweighted Least Squares). See notes for the mathematical derivation.
2. Simulate some toy data to check if our solvers give correct solutions.

***
### Ridge Regression (RR)

Given $X \in \mathbb{R}^{n \times p}$ and $y \in \mathbb{R}^n$, solve
$$
\min_{\beta \in \mathbb{R}^p} \frac{1}{n} \|y - X \beta\|^2 + \lambda \|\beta\|^2 \,.
$$

In [ ]:
# Ridge Regression (RR)
def solveRR(y, X, lam):
    n, p = X.shape
    assert (len(y) == n)
    
    # Hint:
    # beta = np.linalg.solve(A, b)
    # Finds solution to the linear system Ax = b
    return (beta)

### Evaluation of our Ridge Regression solver:
#### One-dimensional data
(for visualization purposes)

In [ ]:
np.random.seed(42)
n = 100
X = np.random.rand(n)
beta_star = 0.8
y = X * beta_star + 0.1 * np.random.normal(0, 1, n)

lam = .1
beta_hat = solveRR(y, X[:, None], lam)

print(beta_hat)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,7))
plt.title('Ridge regression, lambda = {}'.format(lam))
plt.scatter(X, y)
plt.plot(X, X * beta_hat, c='r')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.show()

#### Evaluation on p-dimensional data
---

In [ ]:
# Toy data
np.random.seed(42) # for reproducibility
n = 100 # number of samples
p = 10 # number of features
X = np.random.normal(0, 1, (n, p))
X = sklearn.preprocessing.scale(X) # scale to 0 mean and 1 standard deviation
beta_star = np.random.normal(0, 1, p)
# y = X * beta + noise
y = X.dot(beta_star) + 0.2 * np.random.normal(0, 1, n) 

def compare(beta1, beta2):
    print('''
Our solver:
{}
Scikit-learn:
{}

Difference between the two:
{:.1e}
        '''.format(beta1.round(2), beta2.round(2), float(np.sum((beta1-beta2)**2)))
    )

In [ ]:
lam = 0.1

# Our solver
beta1 = solveRR(y, X, lam)

# Python solver
alpha = lam * X.shape[0]
model = lm.Ridge(alpha=alpha, fit_intercept=False, normalize=False)
beta2 = model.fit(X, y).coef_

# Check
compare(beta1, beta2)

***
### Weighted Ridge Regression (WRR)

Given $X \in \mathbb{R}^{n \times p}$ and $y \in \mathbb{R}^n$, and weights $w \in \mathbb{R}^n_+$, solve
$$
\min_{\beta \in \mathbb{R}^p} \frac{1}{n} \sum_{i=1}^n w_i (y_i - \beta^\top x_i)^2 + \lambda \|\beta\|^2 \,.
$$

**Goal:** Express the objective as a regular Ridge Regression (RR)

In [ ]:
# Weighted Ridge Regression (WRR)
def solveWRR(y, X, w, lam):
    n, p = X.shape
    assert (len(y) == len(w) == n)

    # Hint:
    # Find y1 and X1 such that:
    # beta = solveRR(y1, X1, lam)
    return (beta)

**Try it out:**

In [ ]:
lam = 0.1
w = np.random.rand(len(y))

# Our solver
beta1 = solveWRR(y, X, w, lam)

# Python solver
alpha = lam * X.shape[0]
model = lm.Ridge(alpha=alpha, fit_intercept=False, normalize=False)
beta2 = model.fit(X, y, sample_weight=w).coef_

# Check
compare(beta1, beta2)

***
## Logistic Ridge Regression (LRR)
----

Given $X \in \mathbb{R}^{n \times p}$ and $y \in \{-1,+1\}^n$, solve
$$
\min_{\beta \in \mathbb{R}^p} \frac{1}{n} \sum_{i=1}^n \log (1+e^{-y_i \beta^\top x_i}) + \lambda \|\beta\|^2 \,.
$$

Let $\sigma(x) = \frac{1}{1 + e^{-x}}$ be the sigmoid function.

**Exercise:** Compute $\sigma'(x)$

Rewriting $J$:
$$
J(\beta) = - \frac{1}{n} \sum_{i=1}^n {\log(\sigma(y_i\beta^\top x_i))} + \lambda \|\beta\|^2 \,.
$$

**Exercise:** Compute its gradient $\nabla J$, and its Hessian $\nabla^2 J$
$$$$

### Gradient descent

- Initialize $\beta^{old} = \beta_0$
- Repeat until convergence: $$
\beta^{new} \leftarrow \beta^{old} - h \nabla J(\beta^{old})
$$
where $h$ is the step size (learning rate)

Under some conditions on $J$ (for example $J$ is strongly convex), and with an appropriate step-size $h$, this algorithm converges to a point $\beta^*$ such that $\nabla J(\beta^*) = 0$  

### Implementation of Logistic Ridge Regression with Gradient Descent
---

In [ ]:
# Logistic Ridge Regression (LRR) with gradient descent (GD)
def solveLRR_gradient(y, X, lam, max_iter=500, eps=1e-3):
    '''
    lam: Regularization parameter
    max_iter: Max number of iterations of gradient descent
    eps: Tolerance for stopping criteria 
    '''
    n, p = X.shape
    assert (len(y) == n)
            
    # for i in range(max_iter):
    #     ...
    #         
    return (beta)

**Try it out:**

In [ ]:
y_bin = np.sign(y) # Binarize targets
lam = 0.1

# Our solver
beta_gradient = solveLRR_gradient(y_bin, X, lam)

# Python solver
alpha = 2 * lam * X.shape[0]
model = lm.LogisticRegression(C=1/alpha, fit_intercept=False)
beta_sklearn = model.fit(X, y_bin).coef_

# Check
compare(beta_gradient, beta_sklearn)

### Solving for optimal $\beta$ using Newton-Raphson
$$
\beta^{new} \leftarrow \beta^{old} - \left(\nabla^2 J(\beta^{old})\right)^{-1} \nabla J(\beta^{old})
$$

Show that each step is equivalent to solving a weighted ridge regression (WRR)


<font color='green'>Quadratic approximation to $J$</font>:

$$
J(\beta) \approx J_q(\beta) = J(\beta^{old}) + (\beta - \beta^{old})^\top \nabla J(\beta^{old}) + \frac{1}{2} (\beta - \beta^{old})^\top \nabla^2 J(\beta^{old}) (\beta - \beta^{old})
$$

- **Step 1**: Show that $$\min_\beta J_q(\beta) = \beta^{new}$$


- **Step 2**: Show that $J_q$ is a WRR objective -- find $W$, $z$ such that:
<font color='green'>
$$
2J_q(\beta) = (z - X\beta)^\top W (z - X\beta) + 2\lambda \|\beta\|^2 + \mathrm{C}
$$
</font>

### Implementation of Logistic Ridge Regression with Newton-Raphson method
---

In [ ]:
# Logistic Ridge Regression with Newton-Raphson
def solveLRR_newton(y, X, lam, max_iter=500, eps=1e-3):
    '''
    lam: Regularization parameter
    max_iter: Max number of iterations of gradient descent
    eps: Tolerance for stopping criteria
    '''
    n, p = X.shape
    assert (len(y) == n)
            
    # Hint: Use IRLS
    # for i in range(max_iter):
    #     ...
    #     beta = solveWRR(z, X, w, 2*lam)    
    return (beta)

#### Evaluation

In [ ]:
# Our solver
beta_newton = solveLRR_newton(y_bin, X, lam)

compare(beta_newton, beta_sklearn)

***
### Mini Data Challenge

We will try to predict whether patients have breast cancer.

We use scikit-learn's [breast cancer dataset](https://scikit-learn.org/stable/datasets/index.html#breast-cancer-dataset)

30 features, 569 samples, 2 labels ('malignant' or 'benign')

In [ ]:
# Load data and split into training / validation sets
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# X, y = load_breast_cancer(return_X_y=True)
data = load_breast_cancer()
X, y = data['data'], data['target']
y = 2*y - 1 # transform from {0, 1} to {-1, 1}

# Hint: Scaling can be important
X = sklearn.preprocessing.scale(X)

# Split the dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
X.shape

In [ ]:
# Fit our model and compute its parameters
lam = 0.01
beta = solveLRR_newton(y_train, X_train, lam)

In [ ]:
# Compute predicted probabilities and classes
# probas_pred = ?
# y_pred = ?

#### Performance evaluation
---

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

print("Our model's performance:")
print('Accuracy: {:.2%}'.format(accuracy_score(y_test, y_pred)))
print('AUC: {:.2%}'.format(roc_auc_score(y_test, probas_pred)))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

#### Plotting the feature importances
---

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
features = data['feature_names']
ax.barh(features, beta)
plt.show()